<center><img src="https://github.com/hse-ds/iad-applied-ds/blob/master/2021/hw/hw1/img/logo_hse.png?raw=1" width="1000"></center>

<h1><center>Прикладные задачи анализа данных</center></h1>
<h2><center>Домашнее задание 4: рекомендательные системы</center></h2>

# Введение

В этом задании Вы продолжите работать с данными из семинара [Articles Sharing and Reading from CI&T Deskdrop](https://www.kaggle.com/gspmoreira/articles-sharing-reading-from-cit-deskdrop).

# Загрузка и предобработка данных

In [1]:
import pandas as pd
import numpy as np
import math
import re
import nltk

Загрузим данные и проведем предобраотку данных как на семинаре.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
! cd
! unzip ./drive/MyDrive/dataset/arc1.zip && rm ./drive/MyDrive/dataset/arc1.zip
! unzip ./drive/MyDrive/dataset/arc2.zip && rm ./drive/MyDrive/dataset/arc2.zip

Archive:  ./drive/MyDrive/dataset/arc1.zip
  inflating: articles_embeddings.pickle  
  inflating: articles_metadata.csv   
  inflating: clicks/clicks/clicks_hour_000.csv  
  inflating: clicks/clicks/clicks_hour_001.csv  
  inflating: clicks/clicks/clicks_hour_002.csv  
  inflating: clicks/clicks/clicks_hour_003.csv  
  inflating: clicks/clicks/clicks_hour_004.csv  
  inflating: clicks/clicks/clicks_hour_005.csv  
  inflating: clicks/clicks/clicks_hour_006.csv  
  inflating: clicks/clicks/clicks_hour_007.csv  
  inflating: clicks/clicks/clicks_hour_008.csv  
  inflating: clicks/clicks/clicks_hour_009.csv  
  inflating: clicks/clicks/clicks_hour_010.csv  
  inflating: clicks/clicks/clicks_hour_011.csv  
  inflating: clicks/clicks/clicks_hour_012.csv  
  inflating: clicks/clicks/clicks_hour_013.csv  
  inflating: clicks/clicks/clicks_hour_014.csv  
  inflating: clicks/clicks/clicks_hour_015.csv  
  inflating: clicks/clicks/clicks_hour_016.csv  
  inflating: clicks/clicks/clicks_hour_017.c

In [13]:
!ls

articles_embeddings.pickle  clicks_sample.csv  shared_articles.csv
articles_metadata.csv	    drive	       users_interactions.csv
clicks			    sample_data


In [14]:
articles_df = pd.read_csv("./shared_articles.csv")
articles_df = articles_df[articles_df["eventType"] == "CONTENT SHARED"]
articles_df.head(2)

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en


In [15]:
interactions_df = pd.read_csv("./users_interactions.csv")
interactions_df.head(2)

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US


In [16]:
interactions_df.personId = interactions_df.personId.astype(str)
interactions_df.contentId = interactions_df.contentId.astype(str)
articles_df.contentId = articles_df.contentId.astype(str)

In [17]:
# зададим словарь определяющий силу взаимодействия
event_type_strength = {
   "VIEW": 1.0,
   "LIKE": 2.0, 
   "BOOKMARK": 2.5, 
   "FOLLOW": 3.0,
   "COMMENT CREATED": 4.0,  
}

interactions_df["eventStrength"] = interactions_df.eventType.apply(lambda x: event_type_strength[x])

Оставляем только тех пользователей, которые произамодействовали более чем с пятью статьями.

In [18]:
users_interactions_count_df = (
    interactions_df
    .groupby(["personId", "contentId"])
    .first()
    .reset_index()
    .groupby("personId").size())
print("# users:", len(users_interactions_count_df))

users_with_enough_interactions_df = \
    users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[["personId"]]
print("# users with at least 5 interactions:",len(users_with_enough_interactions_df))

# users: 1895
# users with at least 5 interactions: 1140


Оставляем только те взаимодействия, которые относятся к отфильтрованным пользователям.

In [19]:
interactions_from_selected_users_df = interactions_df.loc[np.in1d(interactions_df.personId,
            users_with_enough_interactions_df)]

In [20]:
print(f"# interactions before: {interactions_df.shape}")
print(f"# interactions after: {interactions_from_selected_users_df.shape}")

# interactions before: (72312, 9)
# interactions after: (69868, 9)


Объединяем все взаимодействия пользователя по каждой статье и сглажиываем полученный результат, взяв от него логарифм.

In [21]:
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = (
    interactions_from_selected_users_df
    .groupby(["personId", "contentId"]).eventStrength.sum()
    .apply(smooth_user_preference)
    .reset_index().set_index(["personId", "contentId"])
)
interactions_full_df["last_timestamp"] = (
    interactions_from_selected_users_df
    .groupby(["personId", "contentId"])["timestamp"].last()
)
        
interactions_full_df = interactions_full_df.reset_index()
interactions_full_df.head(5)

,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.000000,1470395911
1,-1007001694607905623,-6623581327558800021,1.000000,1487240080
2,-1007001694607905623,-793729620925729327,1.000000,1472834892
3,-1007001694607905623,1469580151036142903,1.000000,1487240062
4,-1007001694607905623,7270966256391553686,1.584963,1485994324


Разобьём выборку на обучение и контроль по времени.

In [22]:
from sklearn.model_selection import train_test_split

split_ts = 1475519530
interactions_train_df = interactions_full_df.loc[interactions_full_df.last_timestamp < split_ts].copy()
interactions_test_df = interactions_full_df.loc[interactions_full_df.last_timestamp >= split_ts].copy()

print(f"# interactions on Train set: {len(interactions_train_df)}")
print(f"# interactions on Test set: {len(interactions_test_df)}")

interactions_train_df

# interactions on Train set: 29329
# interactions on Test set: 9777


,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.0,1470395911
2,-1007001694607905623,-793729620925729327,1.0,1472834892
6,-1032019229384696495,-1006791494035379303,1.0,1469129122
7,-1032019229384696495,-1039912738963181810,1.0,1459376415
8,-1032019229384696495,-1081723567492738167,2.0,1464054093
...,...,...,...,...
39099,997469202936578234,9112765177685685246,2.0,1472479493
39100,998688566268269815,-1255189867397298842,1.0,1474567164
39101,998688566268269815,-401664538366009049,1.0,1474567449
39103,998688566268269815,6881796783400625893,1.0,1474567675


Для удобства подсчёта качества запишем данные в формате, где строка соответствует пользователю, а столбцы будут истинными метками и предсказаниями в виде списков.

In [23]:
interactions = (
    interactions_train_df
    .groupby("personId")["contentId"].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={"contentId": "true_train"})
    .set_index("personId")
)

interactions["true_test"] = (
    interactions_test_df
    .groupby("personId")["contentId"].agg(lambda x: list(x))
)

# заполнение пропусков пустыми списками
interactions.loc[pd.isnull(interactions.true_test), "true_test"] = [
    "" for x in range(len(interactions.loc[pd.isnull(interactions.true_test), "true_test"]))]

interactions.head(1)

,true_train,true_test
personId,,
-1007001694607905623,"[-5065077552540450930, -793729620925729327]","[-6623581327558800021, 1469580151036142903, 72..."


# Библиотека LightFM

Для рекомендации Вы будете пользоваться библиотекой [LightFM](https://making.lyst.com/lightfm/docs/home.html), в которой реализованы популярные алгоритмы. Для оценивания качества рекомендации, как и на семинаре, будем пользоваться метрикой *precision@10*.

In [24]:
!pip install lightfm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 310 kB 13.6 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705363 sha256=24cd55e4da0e0a65647a42472595512ddf5113866d321d64a3031121583fcf1e
  Stored in directory: /root/.cache/pip/wheels/f8/56/28/5772a3bd3413d65f03aa452190b00898b680b10028a1021914
Successfully built lightfm


In [25]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k

## Задание 1 (2 балла)

Модели в LightFM работают с разреженными матрицами. Создайте разреженные матрицы `data_train` и `data_test` (размером количество пользователей на количество статей), такие что на пересечении строки пользователя и столбца статьи стоит сила их взаимодействия, если взаимодействие было, и стоит ноль, если взаимодействия не было.

In [26]:
# Ваш код здесь
from scipy.sparse import csr_matrix

def sp_matrix(data, users, contents):
    
    person_ids = data['personId'].values
    content_ids = data['contentId'].values
    event_strengths = data['eventStrength'].values
    
    matr = pd.DataFrame(0, index=users, columns=contents)
    
    for i in range(data.shape[0]):
        if event_strengths[i] is np.nan:
            matr.loc[person_ids[i], content_ids[i]] = 0
        else:
            matr.loc[person_ids[i], content_ids[i]] = event_strengths[i] 
    sp_matr = csr_matrix(matr.values)
    # Удалим nan'ы из матрицы
    np.isnan(sp_matr.data).any()
    sp_matr.data = np.nan_to_num(sp_matr.data)
    sp_matr.eliminate_zeros()
    
    return sp_matr

all_content_ids = set(pd.concat([interactions_train_df['contentId'], interactions_test_df['contentId']]))
all_person_ids = set(pd.concat([interactions_train_df['personId'], interactions_test_df['personId']]))

data_train = sp_matrix(interactions_train_df,all_person_ids, all_content_ids)
data_test = sp_matrix(interactions_test_df,all_person_ids, all_content_ids)

In [27]:
data_train, data_test

(<1140x2984 sparse matrix of type '<class 'numpy.float64'>'
 	with 29329 stored elements in Compressed Sparse Row format>,
 <1140x2984 sparse matrix of type '<class 'numpy.float64'>'
 	with 9777 stored elements in Compressed Sparse Row format>)

## Задание 2 (1 балл)

Обучите модель LightFM с `loss="warp"` и посчитайте *precision@10* на тесте.

In [28]:
# Ваш код здесь
model = LightFM(loss="warp")
model.fit(data_train, epochs=20)

train_prec = precision_at_k(model, data_train, k=10)
test_prec = precision_at_k(model, data_test, k=10, 
                           train_interactions=data_train)

print("Precision on train:", train_prec.mean())
print("Precision on test:", test_prec.mean())

Precision on train: 0.226259
Precision on test: 0.0069246436


## Задание 3 (3 балла)

При вызове метода `fit` LightFM позволяет передавать в `item_features` признаковое описание объектов. Воспользуемся этим. Будем получать признаковое описание из текста статьи в виде [TF-IDF](https://ru.wikipedia.org/wiki/TF-IDF) (можно воспользоваться `TfidfVectorizer` из scikit-learn). Создайте матрицу `feat` размером количесвто статей на размер признакового описание и обучите LightFM с `loss="warp"` и посчитайте precision@10 на тесте.

In [29]:
# Ваш код здесь
from sklearn.feature_extraction.text import TfidfVectorizer

tfid = TfidfVectorizer()
feat = tfid.fit_transform(articles_df['text'])

In [30]:
model2 = LightFM(loss="warp")
model2.fit(data_train, item_features=feat, epochs=20)

train_prec2 = precision_at_k(model2, data_train, item_features=feat, k=10)
test_prec2 = precision_at_k(model2, data_test, item_features=feat, k=10, 
                            train_interactions=data_train)

print("Precision on train:", train_prec2.mean())
print("Precision on test:", test_prec2.mean())

Precision on train: 0.23084533
Precision on test: 0.0069246436


## Задание 4 (2 балла)

В задании 3 мы использовали сырой текст статей. В этом задании необходимо сначала сделать предобработку текста (привести к нижнему регистру, убрать стоп слова, привести слова к номральной форме и т.д.), после чего обучите модель и оценить качество на тестовых данных.

In [31]:
# Ваш код здесь
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords 
nltk.download('stopwords')
stop_words = stopwords.words('english')
porter = PorterStemmer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [32]:
def preprocess_text(text):    
    words = re.sub(r"[^A-Za-z0-9\-]", " ", text).lower().split()
    for word in words:
        if word not in stop_words:
            words = [porter.stem(word)]
    return words

In [33]:
tfid2 = TfidfVectorizer(analyzer='word', tokenizer=preprocess_text)
feat2 = tfid2.fit_transform(articles_df['text'])

model3 = LightFM(loss="warp")
model3.fit(data_train, item_features=feat2, epochs=20)

In [34]:
train_prec3 = precision_at_k(model3, data_train, item_features=feat2, k=10)
test_prec3 = precision_at_k(model3, data_test, item_features=feat2, k=10, 
                            train_interactions=data_train)

print("Precision on train:", train_prec3.mean())
print("Precision on test:", test_prec3.mean())

Precision on train: 0.16852519
Precision on test: 0.0052953158


Улучшилось ли качество предсказания?

Нет, стало только хуже.

## Задание 5 (2 балла)

Подберите гиперпараметры модели LightFM (`n_components` и др.) для улучшения качества модели.

## Бонусное задание (3 балла)

Выше мы использовали достаточно простое представление текста статьи в виде TF-IDF. В этом задании Вам нужно представить текст статьи (можно вместе с заголовком) в виде эмбеддинга полученного с помощью рекуррентной сети или трансформера (можно использовать любую предобученную модель, которая Вам нравится). Обучите модель с ипользованием этих эмеддингов и сравните результаты с предыдущими.

In [ ]:
# Ваш код здесь